In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import requests
import re
import pandas as pd
import numpy as np
from datetime import date, timedelta, datetime
from geopy.geocoders import Nominatim
import time

In [2]:
opciones=Options()

opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)

opciones.headless=False    # si True, no aperece la ventana (headless=no visible)

opciones.add_argument('--start-maximized')         # comienza maximizado

#opciones.add_argument('--incognito')              # incognito

PATH= './chromedriver'
driver = webdriver.Chrome(PATH, options=opciones)

In [3]:
driver.get('https://www.atrapalo.com/hoteles')

In [4]:
driver.find_element_by_class_name("default-input.tt-input").click()


In [5]:
driver.find_element_by_class_name("default-input.tt-input").send_keys("paris")

In [6]:
driver.find_element_by_class_name("boton-buscar.form-submit.columns.large-1.large-uncentered.show-for-large").click()

In [7]:
time.sleep(10)
#driver.find_element_by_xpath('//*[@id="sortingContainer"]/nav/div[2]/ul/li[2]').click()

In [8]:
texto_columnas = driver.find_element_by_class_name('hotels-list.box-resultados')

## Sacar Hoteles

In [9]:
nombre = []
while len(nombre) == 0:
    time.sleep(5)
    texto_hoteles = driver.find_elements_by_class_name('openFontSemiBold')
    time.sleep(5)
    texto_hoteles = texto_hoteles[:4]
    
    for i in texto_hoteles:
            nombre.append(i.text)
    break

## Sacar Precio

In [10]:
time.sleep(6)
texto_precio = texto_columnas.find_elements_by_class_name('value')
texto_precio = texto_precio[0:4]

In [11]:
precio = []
for i in texto_precio:
    precio.append(i.text)

## Sacar dirección

In [12]:
hotel = driver.find_elements_by_class_name('product-name')

In [13]:
direcc = []
for i in hotel:
    primero = i.get_attribute("href")
    direcc.append(primero)

In [14]:
direcc = direcc[0:4]

In [15]:
driver.quit()

In [16]:
time.sleep(5)
address = []
for link in direcc:
    driver = webdriver.Chrome("./chromedriver")
    driver.get(link)
    time.sleep(3)
    calle = driver.find_element_by_css_selector("span.app-address")
    calle = calle.text
    address.append(calle)
    driver.quit()

In [25]:
address

['61 Quai de Grenelle, Paris 75015, Paris Ile de France (Francia) ',
 '4 Rue Des Innocents, Paris 75001, Paris Ile de France (Francia) ',
 '10 rue du Conservatoire, Paris 75009, Paris Ile de France (Francia) ',
 '21 Rue De Chabrol, Paris 75010, Paris Ile de France (Francia) ']

In [29]:
latitud = []
longitud = []
for i in address:
    geolocator = Nominatim(user_agent="edu", timeout=5)
    location = geolocator.geocode(i)
    lat = location.latitude
    lon = location.longitude
    latitud.append(lat)
    longitud.append(lon)

In [30]:
df = pd.DataFrame(list(zip(nombre,precio,address,latitud,longitud)), columns = ['Nombre','Precio','Direccion','Latitud','Longitud'])
df

,Nombre,Precio,Direccion,Latitud,Longitud
0,Hotel Novotel Paris Tour Eiffel,199€,"61 Quai de Grenelle, Paris 75015, Paris Ile de...",48.849660,2.283704
1,Aparthotel Citadines Les Halles,99€,"4 Rue Des Innocents, Paris 75001, Paris Ile de...",48.860496,2.347384
2,Hotel Peyris Opera,71€,"10 rue du Conservatoire, Paris 75009, Paris Il...",48.873907,2.347058
3,"Hotel The Originals City, Hôtel Parisiana, Par...",66€,"21 Rue De Chabrol, Paris 75010, Paris Ile de F...",48.876662,2.354194


In [26]:
geolocator = Nominatim(user_agent="edu", timeout=5)
location = geolocator.geocode("61 Quai de Grenelle, Paris 75015, Paris Ile de France (Francia)")
lat = location.latitude
lon = location.longitude


In [27]:
lat

48.8496595

In [28]:
lon

2.2837038915934476